In [2]:
import os
import shutil

In [3]:
pwd

'/mmfs1/data/aglinska/BC-fMRI-AE/Batch_jobs/ABIDE-II-slurm_jobs'

In [4]:
indir = '../../../data/ABIDE2/RawData/'
folders = [f for f in os.listdir(indir) if f.startswith('ABIDEII')]
folders.sort()
folders

['ABIDEII-BNI_1',
 'ABIDEII-EMC_1',
 'ABIDEII-ETHZ_1',
 'ABIDEII-GU_1',
 'ABIDEII-IP_1',
 'ABIDEII-IU_1',
 'ABIDEII-KKI_1',
 'ABIDEII-KUL_3',
 'ABIDEII-NYU_1',
 'ABIDEII-NYU_2',
 'ABIDEII-OHSU_1',
 'ABIDEII-ONRC_2',
 'ABIDEII-SDSU_1',
 'ABIDEII-TCD_1',
 'ABIDEII-UCD_1',
 'ABIDEII-UCLA_1',
 'ABIDEII-UCLA_Long',
 'ABIDEII-UPSM_Long',
 'ABIDEII-USM_1']

In [5]:
subCount = list()
for idx,folder in enumerate(folders):
    nsubs = len([cont for cont in os.listdir(os.path.join(indir,folder)) if all((cont.startswith('sub-'),os.path.isdir(os.path.join(indir,folder,cont))))])
    subCount.append(nsubs)
    print(f'{idx:02} | {folder} | {nsubs}')

00 | ABIDEII-BNI_1 | 58
01 | ABIDEII-EMC_1 | 54
02 | ABIDEII-ETHZ_1 | 37
03 | ABIDEII-GU_1 | 106
04 | ABIDEII-IP_1 | 56
05 | ABIDEII-IU_1 | 40
06 | ABIDEII-KKI_1 | 211
07 | ABIDEII-KUL_3 | 28
08 | ABIDEII-NYU_1 | 78
09 | ABIDEII-NYU_2 | 27
10 | ABIDEII-OHSU_1 | 93
11 | ABIDEII-ONRC_2 | 59
12 | ABIDEII-SDSU_1 | 58
13 | ABIDEII-TCD_1 | 42
14 | ABIDEII-UCD_1 | 32
15 | ABIDEII-UCLA_1 | 32
16 | ABIDEII-UCLA_Long | 21
17 | ABIDEII-UPSM_Long | 17
18 | ABIDEII-USM_1 | 33


In [6]:
i=-1

In [217]:
i+=1
folder = folders[i]
n = subCount[i]
print(i)
print(folder)
print(n)

IndexError: list index out of range

In [207]:
if not os.path.isdir(f'fmriprep_{folder}'):
    os.mkdir(f'fmriprep_{folder}')

In [208]:
text = '''#!/bin/bash

#SBATCH --job-name=job_fmriprep_{folder}
#SBATCH --output=./fmriprep_{folder}/output_fmriprep_{folder}_%a
#SBATCH --error=./fmriprep_{folder}/error_fmriprep_{folder}_%a
#SBATCH --ntasks=1
#SBATCH --time=10:00:00
#SBATCH --mem=32gb
#SBATCH --array=0-{n}

module load singularity/

date

#rm -rf ./fmriprep_{folder}/output_*
#rm -rf ./fmriprep_{folder}/error_*

cd $HOME

data_dir=$HOME'/data/ABIDE2/RawData/{folder}/'
sing_image=$HOME/fmriprep-22

cd $data_dir
files=(sub*)

cd $HOME

output_dir=$HOME'/data/ABIDE2/Derivatives/fmriprep2/'

fs_lic=$HOME'/fs_licence.txt'

sub=${{files[$SLURM_ARRAY_TASK_ID]}} #;echo $sub


echo $HOME
echo $data_dir
echo $output_dir
echo $sub

singularity run --bind /data/aglinska/scratch:/scratch --cleanenv $sing_image $data_dir $output_dir participant --participant-label $sub --fs-no-reconall --task-id 'rest' --fs-license-file $fs_lic --ignore slicetiming --output-spaces MNI152NLin2009cAsym:res-2


'''

In [209]:
text = text.format(folder=folder,n=n)

In [210]:
with open(f'job_fmriprep_{folder}.slurm', 'w') as f:
    f.write(text)

In [211]:
bashCommand = f'sbatch job_fmriprep_{folder}.slurm'
print(bashCommand)

sbatch job_fmriprep_ABIDEII-USM_1.slurm


In [212]:
with open(f'command.sh', 'w') as f:
    f.write(bashCommand)

In [213]:
!chmod 777 command.sh

In [214]:
!./command.sh

Submitted batch job 335929


In [215]:
#!sbatch job_fmriprep_ABIDEII-ETHZ_1.slurm

In [216]:
#!squeue